# Anomaly Detection

**The problem:** When we started looking into grading buildings by their GHG emissions intensity,
all of the buildings that we were going to be giving an A grade appear to be outliers, missing data
or having faulty data.

The A buildings from our [first PR](https://github.com/vkoves/electrify-chicago/pull/140/commits/14546521270ade9e47623f615af4e6868c6c9cfc) are as follows:

- https://electrifychicago.net/building/1830-n-winchester-ave/ (ID 117024)
  Multi-family housing. Reported 0 natural gas use for the past two years despite non-zero use before.

- https://electrifychicago.net/building/830-n-michigan-ave/ (ID 124236)
  Topshop and UNIQLO building, may be largely vacant, had precipitous declines

- https://electrifychicago.net/building/u-s-cellular-plaza-8430-goby-llc/ (ID 160142)
  Large decline in electricity use (3x from 2017-202), never reported gas use. Could be correct?

- https://electrifychicago.net/building/moody-bible-institute-solheim-center/ (ID 165717)
  Moody's gym, went from 2M kBTUs of natural gas to 0 in 2021 and 2022.

- https://electrifychicago.net/building/newberry-plaza-townhouse-owners-association/ (ID 172137)
  Similarly went to 0 gas from 800k KBTU,

- https://electrifychicago.net/building/u-s-cellular-plaza-8420-goby-llc/ (ID 251770)

- https://electrifychicago.net/building/4434-4444-n-damen-ave/ (ID 254001)
    Robey Condominiums, multi-family housing. Reported 0 natural gas use for the past two years
    despite non-zero use before.


## Dependencies

This notebook requires:

- pandas
- numpy
- plotly
- statsmodels
- nbformat

To install, _in this `src/data/analysis` directory_, run:

```
pip install -r requirements.txt
```

In [2]:
import pandas as pd
import numpy as np
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot
import plotly.io as pio
from plotly.subplots import make_subplots
import statsmodels.api as sm
import os
from pathlib import Path
import json

from IPython.display import Image

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

pd.set_option('display.max_columns', None)

## Set pathing

In [3]:
# get static dir for saving images
current_dir = Path.cwd()
project_root = current_dir

while True:
    if os.path.basename(project_root) == 'electrify-chicago':
        print("Success: Found 'electrify-chicago' as the base directory.")
        break
    new_root = os.path.dirname(project_root)
    if new_root == project_root:  # Reached the filesystem root
        raise FileNotFoundError("Error: 'electrify-chicago' directory not found in the path hierarchy.")
    project_root = new_root
static_blog_pth = os.path.join(project_root, 'static', 'blog', 'GHGIntensityPredictCompliance')
os.makedirs(static_blog_pth, exist_ok=True)

expected_dir_name = "analysis"
fig_dir = os.path.join(current_dir, 'output', 'compliance_analysis')

# Check if the current directory is the "analysis" folder
if current_dir.name != expected_dir_name:
    raise AssertionError(f"Expected working directory to be '{expected_dir_name}', but got '{current_dir.name}'.\n"
                         f"Please ensure you are in the correct directory.")

print(f"Current working directory is correctly set to '{current_dir}'.")

Success: Found 'electrify-chicago' as the base directory.
Current working directory is correctly set to '/home/viktor/Documents/electrify-chicago/src/data/analysis'.


### Notebook options and custom plotting function

In [4]:
reduce_memory = True # option to display plotly as static images to reduce memory, if possible
export_to_blog = False # if true, saves plots and regressions to blog static folder for website publishing

if export_to_blog:
    dirs = [static_blog_pth, fig_dir]
else:
    dirs = [fig_dir]

def show_fig(fig, reduce_memory):
    if reduce_memory:
        try:
            png_image = pio.to_image(fig, format='png')
            return (png_image, reduce_memory)

        except:
            print("Error exporting plotly to png, displaying html graph instead")
            reduce_memory = False

    if not reduce_memory:
        return (iplot(fig), reduce_memory)


## Read in data

In [5]:
# Construct the path to the CSV file (one level above the current directory)
data_path = os.path.join( current_dir.parent, 'dist', 'benchmarking-all-years.csv')

df = pd.read_csv(data_path)

# Create the "reported" column
df['Reported'] = df['GHGIntensity'].notna().astype(int)

print(f"There are {df['ID'].unique().shape[0]} unique building ids")

df['DataYear'] = df['DataYear'].astype(int)

df.head()

There are 3749 unique building ids


,ID,DataYear,ReportingStatus,GrossFloorArea,TotalGHGEmissions,GHGIntensity,NumberOfBuildings,ChicagoEnergyRating,ENERGYSTARScore,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,SiteEUI,SourceEUI,WeatherNormalizedSiteEUI,WeatherNormalizedSourceEUI,Reported
0,252064,2020,Submitted Data,64028.0,1160.9,18.1,1.0,2.0,NaN,2384738.9,NaN,7438787.0,5594040.1,NaN,240.8,323.6,246.0,329.9,1
1,232458,2020,Submitted Data,627680.0,4871.7,7.8,1.0,1.0,22.0,16397682.8,43537490.6,NaN,NaN,NaN,95.5,146.0,100.3,150.7,1
2,254616,2020,Submitted Data,555524.0,4581.4,8.2,1.0,2.0,49.0,28606427.7,2199940.1,NaN,NaN,NaN,55.5,148.3,56.7,151.8,1
3,103812,2020,Submitted Data,130007.0,1092.1,8.4,1.0,3.0,61.0,6489281.3,1493523.2,NaN,NaN,NaN,61.4,151.8,63.0,154.8,1
4,254073,2020,Submitted Data,83000.0,295.8,3.6,1.0,4.0,100.0,1614582.3,825006.6,NaN,NaN,NaN,29.4,64.9,29.6,64.3,1


## Read in Building Benchmark Data to get Building Names

In [6]:
names_path = os.path.join( current_dir.parent, 'dist', 'building-benchmarks.csv')

building_names = pd.read_csv(names_path)[['ID', 'PropertyName' ]]
building_names.drop_duplicates(keep='first')
building_names.head()

,ID,PropertyName
0,100001,Presence SMEMC St Elizabeth Campus
1,100002,Clemente Community Academy HS -CPS
2,100019,Dixon Building
3,100068,Joffco Square
4,100148,The Jeffery Cyril Building


## Merge names to data

In [7]:
df = pd.merge( df, building_names, how='left', on='ID')
df['PropertyName'] = df['PropertyName'].fillna("[Building Name Unavailable]").replace("", "[Building Name Unavailable]")
df = df[df['ReportingStatus'].isin(['Submitted Data'])]
df.head()

,ID,DataYear,ReportingStatus,GrossFloorArea,TotalGHGEmissions,GHGIntensity,NumberOfBuildings,ChicagoEnergyRating,ENERGYSTARScore,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,SiteEUI,SourceEUI,WeatherNormalizedSiteEUI,WeatherNormalizedSourceEUI,Reported,PropertyName
0,252064,2020,Submitted Data,64028.0,1160.9,18.1,1.0,2.0,NaN,2384738.9,NaN,7438787.0,5594040.1,NaN,240.8,323.6,246.0,329.9,1,Mansueto Library
1,232458,2020,Submitted Data,627680.0,4871.7,7.8,1.0,1.0,22.0,16397682.8,43537490.6,NaN,NaN,NaN,95.5,146.0,100.3,150.7,1,Harper Square Cooperative
2,254616,2020,Submitted Data,555524.0,4581.4,8.2,1.0,2.0,49.0,28606427.7,2199940.1,NaN,NaN,NaN,55.5,148.3,56.7,151.8,1,Former Coyne College
3,103812,2020,Submitted Data,130007.0,1092.1,8.4,1.0,3.0,61.0,6489281.3,1493523.2,NaN,NaN,NaN,61.4,151.8,63.0,154.8,1,400 W Superior St
4,254073,2020,Submitted Data,83000.0,295.8,3.6,1.0,4.0,100.0,1614582.3,825006.6,NaN,NaN,NaN,29.4,64.9,29.6,64.3,1,Blue Moon Lofts


### Check that every building/year combo exists only once

In [8]:
group_counts = df.groupby(['ID', 'DataYear']).size()

# Assert that the maximum count in any group is at most 1
assert group_counts.max() <= 1, "There are buildings with more than one row in a given year!"

### Get the latest year we have data for 

In [9]:
# get buildings with zero natural gas use in past year
latestYear = df['DataYear'].max()
latestYear

2022

In [10]:
latestData = df[df['DataYear'] == latestYear]
latestData.head()

,ID,DataYear,ReportingStatus,GrossFloorArea,TotalGHGEmissions,GHGIntensity,NumberOfBuildings,ChicagoEnergyRating,ENERGYSTARScore,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,SiteEUI,SourceEUI,WeatherNormalizedSiteEUI,WeatherNormalizedSourceEUI,Reported,PropertyName
17753,175891,2022,Submitted Data,172500.0,452.4,3.0,1.0,4.0,100.0,2069532.4,3384519.4,0.0,0.0,NaN,36.4,62.3,36.4,62.3,1,[Building Name Unavailable]
17755,251245,2022,Submitted Data,249095.0,1434.0,5.8,2.0,3.5,74.0,3345590.2,18702028.9,0.0,0.0,NaN,88.5,116.4,94.1,121.1,1,3800 N. Lake Shore Drive
21284,256658,2022,Submitted Data,393938.0,1948.4,4.9,8.0,4.0,75.0,6388293.5,20841006.3,0.0,0.0,NaN,69.1,101.0,71.9,102.6,1,Midpointe Apartments
21285,250062,2022,Submitted Data,66285.0,525.4,7.9,1.0,2.0,NaN,2649529.2,3322169.7,0.0,0.0,NaN,90.1,164.5,91.9,165.6,1,RJ Quinn Academy
21286,101545,2022,Submitted Data,51163.0,300.3,5.9,1.0,2.0,50.0,541560.1,4310877.6,0.0,0.0,NaN,94.8,118.1,101.2,123.8,1,[Building Name Unavailable]


In [11]:
noGasUse = latestData[latestData['NaturalGasUse'].isin([0, np.nan])]
noGasUse.head()

,ID,DataYear,ReportingStatus,GrossFloorArea,TotalGHGEmissions,GHGIntensity,NumberOfBuildings,ChicagoEnergyRating,ENERGYSTARScore,ElectricityUse,NaturalGasUse,DistrictSteamUse,DistrictChilledWaterUse,AllOtherFuelUse,SiteEUI,SourceEUI,WeatherNormalizedSiteEUI,WeatherNormalizedSourceEUI,Reported,PropertyName
21287,160438,2022,Submitted Data,1484327.0,9938.8,7.1,1.0,4.0,75.0,63482520.5,0.0,0.0,29929202.7,NaN,66.4,145.8,64.4,144.9,1,155 North Wacker
21297,100179,2022,Submitted Data,450612.0,NaN,NaN,1.0,0.0,NaN,15191302.2,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,Saint Anthony Hospital - Main Hospital
21322,156942,2022,Submitted Data,201402.0,1318.2,6.5,1.0,3.0,68.0,2852269.0,0.0,14194821.6,0.0,NaN,84.6,124.4,89.7,130.6,1,Burton-Judson Courts
21354,250147,2022,Submitted Data,186957.7,1766.0,9.4,1.0,1.0,NaN,13408061.5,0.0,0.0,0.0,NaN,71.7,200.8,71.7,200.8,1,820 W Jackson Blvd
21366,101867,2022,Submitted Data,641962.0,4402.2,6.9,1.0,3.0,67.0,33422010.0,0.0,0.0,0.0,NaN,52.1,145.8,52.9,148.2,1,125 South Wacker


### Get Count of "Gas Free" Buildings Latest Year

In [12]:
noGasUse['ID'].count()

235

### Loop Through Gas Free Buildings And See If They Used Gas in Previous Years

In [13]:
noGasUseIds = noGasUse['ID']
usedGasBefore = df[df['DataYear'] < latestYear][df['NaturalGasUse'] > 0][df['ID'].isin(noGasUseIds)]
gasAnomalyIds = usedGasBefore['ID'].unique()
gasAnomalyIds

/tmp/ipykernel_23063/3249074751.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/tmp/ipykernel_23063/3249074751.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



array([254001, 165664, 165819, 159892, 260149, 175817, 242935, 256639,
       174228, 175914, 135050, 103602, 102987, 113670, 172256, 115942,
       175754, 254378, 175995, 116550, 260135, 175492, 159425, 100179,
       172145, 137144, 101757, 138730, 172565, 101448, 101396, 260116,
       157988, 105441, 252312, 172772, 115605, 165717, 116644, 255944,
       172540, 172157, 159423, 175885, 251883, 238480, 117179, 175334,
       172393, 251959, 175882, 160137, 254035, 254387, 255012, 242279,
       162325, 256614, 260101, 129344])

### Did this catch our A buildings?

In [14]:
AGradedBuildingIds = [117024, 124236, 160142, 165717, 172137, 251770, 254001]
AGradedBuildingIds = set(AGradedBuildingIds)
AGradedBuildingIds.intersection(gasAnomalyIds)

{165717, 254001}

---

## Analysis: Unique values for GHG Intensity

Conclusion: strange that some values are highly represented while others are not. How are these calculated? 

In [15]:
# Round GHG Intensity values to 1 digit
df['GHGIntensity'] = df['GHGIntensity'].round(1)

## Plot Distribution of GHG Intensities

In [16]:

fig = sp.make_subplots(
    rows=1,
    cols=2,
    column_widths=[0.8, 0.2],
    horizontal_spacing=0.2,
    subplot_titles = [
        '',
        '<i style="font-size:14px">GHG Intensity Outliers</i><br><span style="font-size:9px">(GHG Intensity values > 50)</span>'
    ]
)


fig.add_trace( go.Histogram(
    x=df['GHGIntensity'],
    #nbinsx=int((df['GHGIntensity'].max() - df['GHGIntensity'].min()) / 0.5),
    xbins = dict(start=0, end=100, size=.5),
    name='Histogram of Values',
    hovertemplate = " %{y} Buildings <br>with GHG Intensity between %{x}<extra></extra>"
        ),
        row=1,col=1
)


# Add a light red box to the first plot
fig.add_shape(
    type="rect",
    x0=50,
    x1=100,
    y0=0,
    y1=1500,
    fillcolor="rgba(255, 0, 0, 0.1)",  # Light red with transparency
    line=dict(width=0),
    row=1, col=1
)

outlier_subset = df.dropna(subset=['GHGIntensity'])
outlier_subset = outlier_subset[outlier_subset['GHGIntensity']>50]
fig.add_trace(
    go.Scatter(
        x=[0] * len(outlier_subset['GHGIntensity']),  # Make x an array of zeros with the correct length
        y=outlier_subset['GHGIntensity'],
        mode='markers',
        marker=dict( color='blue', opacity=0.6),
        customdata=df['DataYear'],
        hovertext=outlier_subset['PropertyName'],  # Add PropertyName to hovertext
        hovertemplate="%{hovertext}<br>GHG Intensity: %{y} in %{customdata}",
        name=''
    ),
    row=1, col=2
)

# Add a light red background to the second subplot
fig.add_shape(
    type="rect",
    x0=-1,
    x1=1,
    y0=50,
    y1=900,
    fillcolor="rgba(255, 0, 0, 0.1)",  # Light red with transparency
    line=dict(width=0),
    layer="below",
    row=1, col=2
)


fig.update_xaxes(visible=False, row=1, col=2)
fig.update_xaxes(range=[0, 100], row=1,col=1)
fig.update_xaxes( title_text='' , row=1,col=2)
fig.update_yaxes( title_text='GHG Intensity', row=1,col=2)

## Add an outline to the bars
fig.update_traces(marker=dict(line=dict(width=.1, color='black')))

fig.add_annotation(
    x=80,
    y=300,
    text="<i>Some buildings had  <br>outlier GHG intensity <br> levels (up to 800) →</i>",
    showarrow=False,  # No arrow for this annotation
    font=dict(size=10),  # Customize font size
)

# Update layout for better display
fig.update_layout(
    xaxis_title='GHG Intensity',
    yaxis_title='Count',
    showlegend=False,
    title='Distribution of GHG Intensities',
    height=400,
    width=800
)

# Show the plot
#pio.show(fig)
iplot(fig)

for dir in [static_blog_pth, fig_dir]:
    fig.write_html( os.path.join(dir,'distribution_of_GHG_intensity.html'), include_plotlyjs="cdn" )

FileNotFoundError: [Errno 2] No such file or directory: '/home/viktor/Documents/electrify-chicago/src/data/analysis/output/compliance_analysis/distribution_of_GHG_intensity.html'

## Compliance type counts over time

In [ ]:
value_counts.head()

GHGIntensity
0.0    2
0.1    1
0.2    4
0.3    8
0.4    5
Name: count, dtype: int64

In [ ]:
# Count each new column per year
value_counts = df.groupby('DataYear')['Reported'].value_counts()
non_reporting_counts = value_counts.xs(0, level='Reported')
reporting_counts = value_counts.xs(1, level='Reported')

# Create the figure
fig = go.Figure()

# Add traces for each category
fig.add_trace(go.Scatter(x=reporting_counts.index, y=reporting_counts.values,
                         mode='lines+markers', name='Reported',
                         line=dict(width=4, color='rgba(0, 0,255, 0.7)'),
                         marker=dict(symbol='circle', size=10)))

fig.add_trace(go.Scatter(x=non_reporting_counts.index, y=non_reporting_counts.values,
                         mode='lines+markers', name="Didn't Report",
                         line=dict(width=4, color='rgba(255, 0, 0, 0.7)'),
                         marker=dict(symbol='circle', size=10)))

fig.add_trace( go.Scatter( x=[2018.5, 2019.5, 2019.5, 2018.5, 2018.5],
                          y=[0, 0, np.max(reporting_counts)*1.2, np.max(reporting_counts)*1.2, 0 ],
                         fill='toself', mode='lines', name='Covid Data Disruption' )
)

# Update layout
fig.update_layout(title="Count of Buildings That Did/Didn't Report Emissions by Year",
                  xaxis_title='Year of Emissions<br>(One year before data is reported)',
                  yaxis_title='Count of Buildings',
                  legend_title='Category')

# Show the plot
#pio.show(fig)
iplot(fig)


for dir in [static_blog_pth, fig_dir]:
    fig.write_html( os.path.join(dir,'reporting_counts_over_time.html'), include_plotlyjs="cdn" )


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed